In [1]:
import json
import sys,os
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys

sys.path.extend(['/root/xiaoda/query_topic/'])

In [3]:
import torch
from torch.nn import functional as F
import numpy as np
import random
import torch.nn as nn
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np

"""
https://github.com/ondrejbohdal/meta-calibration/blob/main/Metrics/metrics.py
"""

class ECE(nn.Module):
    
    def __init__(self, n_bins=15):
        """
        n_bins (int): number of confidence interval bins
        """
        super(ECE, self).__init__()
        bin_boundaries = torch.linspace(0, 1, n_bins + 1)
        self.bin_lowers = bin_boundaries[:-1]
        self.bin_uppers = bin_boundaries[1:]

    def forward(self, logits, labels, mode='logits'):
        if mode == 'logits':
            softmaxes = F.softmax(logits, dim=1)
        else:
            softmaxes = logits
        # softmaxes = F.softmax(logits, dim=1)
        confidences, predictions = torch.max(softmaxes, 1)
        accuracies = predictions.eq(labels)
        
        ece = torch.zeros(1, device=logits.device)
        for bin_lower, bin_upper in zip(self.bin_lowers, self.bin_uppers):
            # Calculated |confidence - accuracy| in each bin
            in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
            prop_in_bin = in_bin.float().mean()
            if prop_in_bin.item() > 0:
                accuracy_in_bin = accuracies[in_bin].float().mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

        return ece

In [3]:
from topic_risk_query_onnx import TorchInfer, OnnxInfer

In [4]:
green_green_topic_risk_api = TorchInfer('./risk_data_tiny/config_topic_risk_green_v1.ini')

topic /data/albert.xht/raw_chat_corpus/topic_classification_v4/label_list.txt ===schema-path===
senti_query /data/albert.xht/xiaoda/sentiment/senti/senti_query_label.txt ===schema-path===
senti /data/albert.xht/xiaoda/sentiment/senti/senti_label.txt ===schema-path===
bias /data/albert.xht/xiaoda/sentiment/bias/bias_label.txt ===schema-path===
ciron /data/albert.xht/xiaoda/sentiment/ciron/ciron_label.txt ===schema-path===
intent /data/albert.xht/xiaoda/sentiment/intention_data_v2-1/label.txt ===schema-path===
offensive /data/albert.xht/xiaoda/sentiment/offensive/offensive_label.txt ===schema-path===
query_risk /data/albert.xht/xiaoda/sentiment/query_risk_v12/query_risk_label.txt ===schema-path===
teenager /data/albert.xht/xiaoda/sentiment/teenager//teenager_label.txt ===schema-path===
politics /data/albert.xht/xiaoda/sentiment/green_politics/green_politics_label.txt ===schema-path===
porn /data/albert.xht/xiaoda/sentiment/green_porn_v1/green_porn_label.txt ===schema-path===
abusive /dat

01/29/2023 08:03:28 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/29/2023 08:03:28 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/29/2023 08:03:28 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/29/2023 08:03:28 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/29/2023 08:03:28 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/29/2023 08:03:28 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/29/2023 08:03:28 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/29/2023 08:03:28 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/29/2023 08:03:28 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/29/2023 08:03:28 - INFO - nets.them_classifier - ++RobertaClassifier++ apply stable dropout++
01/29/2023 08:03:28 - INFO - n

In [9]:
model_topic_path = '/data/albert.xht/xiaodao/risk_classification/tiny/topic_v4_update_green_v1_teenager_v1_porn_multitask_raw_filter_senti_query_risk_v12_intent_v2-1_10_no_symbol_senti_query_senta_green_mtdnn_v18/multitask_cls.pth.19'

green_green_topic_risk_api.reload(model_topic_path)

In [8]:
import time
start = time.time()
for i in range(10000):
    resp = green_green_topic_risk_api.predict('你是大晒比')
print(time.time()-start, resp)

[autoreload of topic_risk_query_onnx failed: Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/opt/conda/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/opt/conda/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/root/xiaoda/query_topic/topic_risk_query_onnx.py", line 298, in <module>
    topic_risk_api = OnnxInfer('./risk_data_tiny/config_topic_risk_green_v1.ini', use_fp16=False)
  File "/root/xiaoda/query_topic/topi

56.70577335357666 [{'topic': [['电脑/网络', 0.142822265625], ['幽默滑稽', 0.11566162109375], ['校园生活', 0.08612060546875], ['军事', 0.06732177734375], ['游戏', 0.066650390625]], 'senti_query': [['负向', 0.146728515625], ['中性', 0.7919921875], ['正向', 0.0615234375]], 'senti': [['负向', 0.7001953125], ['正向', 0.2998046875]], 'bias': [['偏见', 0.2440185546875], ['正常', 0.755859375]], 'ciron': [['讽刺', 0.450927734375], ['正常', 0.548828125]], 'intent': [['主观评价/比较/判断', 0.53271484375], ['寻求建议/帮助', 1.0848045349121094e-05], ['其它', 0.46728515625]], 'offensive': [['冒犯', 0.079833984375], ['正常', 0.919921875]], 'query_risk': [['风险', 0.6650390625], ['个人信息', 0.00012230873107910156], ['正常', 0.334716796875]], 'teenager': [['不良', 0.1724853515625], ['正常', 0.82763671875]], 'politics': [['涉政', 0.002140045166015625], ['正常', 0.998046875]], 'porn': [['色情', 0.043670654296875], ['低俗', 0.7548828125], ['色情违禁', 0.00014019012451171875], ['正常', 0.201171875]], 'abusive': [['辱骂', 0.4072265625], ['口头语', 0.56298828125], ['正常', 0.0299530029296875]

'cpu'

In [9]:
model_topic_path = '/data/albert.xht/xiaodao/risk_classification/tiny/topic_v4_update_green_v1_teenager_v1_porn_multitask_raw_filter_senti_query_risk_v12_intent_v2-1_10_no_symbol_senti_query_senta_green_mtdnn_v18/multitask_cls.pth.19'
model_onnx_path = '/data/albert.xht/xiaodao/risk_classification/tiny/topic_v4_update_green_v1_teenager_v1_porn_multitask_raw_filter_senti_query_risk_v12_intent_v2-1_10_no_symbol_senti_query_senta_green_mtdnn_v18/onnx/inference.onnx'


In [50]:
import onnxruntime as ort

ort.get_device()

'GPU'

In [49]:
ort.__version__

'1.10.0'

In [4]:
onnx_base_api = OnnxInfer('./risk_data_tiny/config_topic_v5_risk_green_v1_base.ini')

topic /data/albert.xht/raw_chat_corpus/topic_classification_v5/label_list.txt ===schema-path===
senti_query /data/albert.xht/xiaoda/sentiment/senti/senti_query_label.txt ===schema-path===
senti /data/albert.xht/xiaoda/sentiment/senti/senti_label.txt ===schema-path===
bias /data/albert.xht/xiaoda/sentiment/bias/bias_label.txt ===schema-path===
ciron /data/albert.xht/xiaoda/sentiment/ciron/ciron_label.txt ===schema-path===
intent /data/albert.xht/xiaoda/sentiment/intention_data_v2-1/label.txt ===schema-path===
offensive /data/albert.xht/xiaoda/sentiment/offensive/offensive_label.txt ===schema-path===
query_risk /data/albert.xht/xiaoda/sentiment/query_risk_v12/query_risk_label.txt ===schema-path===
teenager /data/albert.xht/xiaoda/sentiment/teenager//teenager_label.txt ===schema-path===
politics /data/albert.xht/xiaoda/sentiment/green_politics/green_politics_label.txt ===schema-path===
porn /data/albert.xht/xiaoda/sentiment/green_porn_v1/green_porn_label.txt ===schema-path===
abusive /dat

In [4]:
from topic_risk_query_onnx import TorchInfer, OnnxInfer
green_green_topic_risk_onnx_api = OnnxInfer('./risk_data_tiny/config_topic_risk_green_v1.ini')

[autoreload of onnxruntime.capi failed: Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/opt/conda/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/opt/conda/lib/python3.8/importlib/__init__.py", line 159, in reload
    raise ImportError(msg.format(parent_name),
ImportError: parent 'onnxruntime' not in sys.modules
]


OSError: libcudnn.so.8: cannot open shared object file: No such file or directory

In [4]:
import time
start = time.time()
for i in range(1):
    resp = green_green_topic_risk_onnx_api.predict(['你是大晒比'])
print(time.time()-start, resp)

0.002851247787475586 [{'topic': [['电脑/网络', 0.142734095454216], ['幽默滑稽', 0.11579287797212601], ['校园生活', 0.08640298247337341], ['军事', 0.06717079877853394], ['游戏', 0.06644245237112045]], 'senti_query': [['负向', 0.14657963812351227], ['中性', 0.7916901111602783], ['正向', 0.06173031032085419]], 'senti': [['负向', 0.6999999284744263], ['正向', 0.30000004172325134]], 'bias': [['偏见', 0.24390576779842377], ['正常', 0.756094217300415]], 'ciron': [['讽刺', 0.449801504611969], ['正常', 0.5501984357833862]], 'intent': [['主观评价/比较/判断', 0.5304533243179321], ['寻求建议/帮助', 1.0922172805294394e-05], ['其它', 0.4695357382297516]], 'offensive': [['冒犯', 0.07948239147663116], ['正常', 0.92051762342453]], 'query_risk': [['风险', 0.6639767289161682], ['个人信息', 0.00012154907017247751], ['正常', 0.3359016180038452]], 'teenager': [['不良', 0.17232348024845123], ['正常', 0.8276764750480652]], 'politics': [['涉政', 0.002137985546141863], ['正常', 0.9978620409965515]], 'porn': [['色情', 0.043555792421102524], ['低俗', 0.7547832131385803], ['色情违禁', 0.000

In [16]:
resp[0]

{'topic': [['电脑/网络', 0.142734095454216],
  ['幽默滑稽', 0.11579287797212601],
  ['校园生活', 0.08640298247337341],
  ['军事', 0.06717079877853394],
  ['游戏', 0.06644245237112045]],
 'senti_query': [['负向', 0.14657963812351227],
  ['中性', 0.7916901111602783],
  ['正向', 0.06173031032085419]],
 'senti': [['负向', 0.6999999284744263], ['正向', 0.30000004172325134]],
 'bias': [['偏见', 0.24390576779842377], ['正常', 0.756094217300415]],
 'ciron': [['讽刺', 0.449801504611969], ['正常', 0.5501984357833862]],
 'intent': [['主观评价/比较/判断', 0.5304533243179321],
  ['寻求建议/帮助', 1.0922172805294394e-05],
  ['其它', 0.4695357382297516]],
 'offensive': [['冒犯', 0.07948239147663116], ['正常', 0.92051762342453]],
 'query_risk': [['风险', 0.6639767289161682],
  ['个人信息', 0.00012154907017247751],
  ['正常', 0.3359016180038452]],
 'teenager': [['不良', 0.17232348024845123], ['正常', 0.8276764750480652]],
 'politics': [['涉政', 0.002137985546141863], ['正常', 0.9978620409965515]],
 'porn': [['色情', 0.043555792421102524],
  ['低俗', 0.7547832131385803],
  ['

In [17]:
resp[1]

{'topic': [['两性', 0.188909113407135],
  ['社会', 0.08938299119472504],
  ['恋爱', 0.06705530732870102],
  ['女性', 0.03403538465499878],
  ['军事', 0.032976988703012466]],
 'senti_query': [['负向', 0.873418927192688],
  ['中性', 0.12284678965806961],
  ['正向', 0.0037343017756938934]],
 'senti': [['负向', 0.9846432209014893], ['正向', 0.015356787480413914]],
 'bias': [['偏见', 0.7863349318504333], ['正常', 0.21366505324840546]],
 'ciron': [['讽刺', 0.36813610792160034], ['正常', 0.6318638324737549]],
 'intent': [['主观评价/比较/判断', 0.9771868586540222],
  ['寻求建议/帮助', 5.9204969147685915e-05],
  ['其它', 0.022753916680812836]],
 'offensive': [['冒犯', 0.9618919491767883], ['正常', 0.038108017295598984]],
 'query_risk': [['风险', 0.9980199337005615],
  ['个人信息', 3.312116632514517e-06],
  ['正常', 0.001976779429242015]],
 'teenager': [['不良', 0.9101048111915588], ['正常', 0.08989515900611877]],
 'politics': [['涉政', 0.00048242579214274883], ['正常', 0.999517560005188]],
 'porn': [['色情', 0.008908435702323914],
  ['低俗', 0.2472759336233139]

In [23]:
import time
start = time.time()
for i in range(1):
    resp = onnx_base_api.predict('你是大晒比阿克索德就放寒假开始的规范化')
print(time.time()-start, resp)

NameError: name 'onnx_base_api' is not defined

In [24]:
import pandas as pd
from tqdm import tqdm
df = pd.read_excel('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/itag_labl_data_deal_20230112.xlsx')
gold, pred = [], []
result_list = []
data_dict = {}
for idx in tqdm(range(df.shape[0])):
    content = df.loc[idx]
    
    score_list = green_green_topic_risk_onnx_api.predict(content['query'])[0]
    if content['query'] not in data_dict:
        data_dict[content['query']] = ''
    else:
        continue
    if content['human_risk'] in [-1]:
        continue
    result_list.append((content, score_list))
    if score_list['query_risk'][0][1] > 0.5 :
        pred.append(1)
    else:
        pred.append(0)
    
    gold.append(int(content['human_risk']))

100%|██████████| 5128/5128 [00:11<00:00, 438.40it/s]


In [14]:
from sklearn.metrics import classification_report

print(classification_report(gold, pred, digits=4))

              precision    recall  f1-score   support

           0     0.9832    0.9203    0.9507      4378
           1     0.6087    0.8873    0.7221       612

    accuracy                         0.9162      4990
   macro avg     0.7960    0.9038    0.8364      4990
weighted avg     0.9372    0.9162    0.9226      4990

